In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time
import os


In [9]:
def obtener_tabla_filtrada_completa(filtro_menu_id, filtro_opcion_id):
    urls = [
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi",
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi?players=51-100"
    ]

    estadisticas = {
        "Serve": "statso",
        "Return": "statsw",
        "Breaks": "statsl",
        "More": "statst"
    }

    options = Options()
    options.add_argument('--headless')
    dfs_por_estadistica = {nombre: [] for nombre in estadisticas}

    for url in urls:
        driver = webdriver.Chrome(options=options)
        try:
            driver.get(url)

            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, filtro_menu_id)))
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "stattab")))

            # Aplicar filtro lateral (ej: superficie)
            driver.find_element(By.ID, filtro_menu_id).click()
            time.sleep(1)
            driver.find_element(By.ID, filtro_opcion_id).click()
            time.sleep(2)

            for nombre_estad, clase in estadisticas.items():
                try:
                    # Cambiar tipo de estadística
                    driver.find_element(By.CLASS_NAME, clase).click()
                    time.sleep(2)

                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    tabla = soup.find("table", {"id": "matches"})
                    if not tabla:
                        continue

                    rows = tabla.find_all("tr")
                    data = []
                    for row in rows:
                        cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                        if cols:
                            data.append(cols)

                    if len(data) <= 1:
                        continue

                    headers = data[0]
                    data_rows = data[1:-1]  # Eliminar la última fila

                    df = pd.DataFrame(data_rows, columns=headers)
                    dfs_por_estadistica[nombre_estad].append(df)

                except Exception as e:
                    print(f"Error con estadística {nombre_estad} en {url}: {e}")

        finally:
            driver.quit()

    # Concatenar los DataFrames por estadística y luego unir por columnas
    dfs_final = []
    for nombre_estad, df_list in dfs_por_estadistica.items():
        if df_list:
            df_concat = pd.concat(df_list, ignore_index=True)
            dfs_final.append(df_concat.reset_index(drop=True))

    if dfs_final:
        df_merged = pd.concat(dfs_final, axis=1)
        return df_merged
    else:
        return pd.DataFrame()

In [10]:
def procesar_filtros_y_guardar(
    filtro_menu_id,
    filtro_id_map,
    carpeta_salida,
    nombre_base
):
    os.makedirs(carpeta_salida, exist_ok=True)

    for nombre_filtro, opcion_id in filtro_id_map.items():
        print(f"Procesando filtro: {nombre_filtro}")
        df = obtener_tabla_filtrada_completa(filtro_menu_id, opcion_id)

        if not df.empty:
            archivo = os.path.join(carpeta_salida, f"{nombre_base}_{nombre_filtro.replace(' ', '_')}.csv")
            df.to_csv(archivo, index=False, encoding="utf-8-sig")
            print(f"Guardado: {archivo}")
        else:
            print(f"No se generó archivo para filtro: {nombre_filtro}")


In [11]:
filtro_id_map_superficies = {
    "Hard": "surface0",
    "Clay": "surface1",
    "Grass": "surface2"
}

procesar_filtros_y_guardar(
    filtro_menu_id="surfacehead",
    filtro_id_map=filtro_id_map_superficies,
    carpeta_salida="ultimas 52 semanas",
    nombre_base="estadisticas_superficies"
)

Procesando filtro: Hard
Guardado: ultimas 52 semanas\estadisticas_superficies_Hard.csv
Procesando filtro: Clay
Guardado: ultimas 52 semanas\estadisticas_superficies_Clay.csv
Procesando filtro: Grass
Guardado: ultimas 52 semanas\estadisticas_superficies_Grass.csv


In [12]:
filtro_id_map_torneos = {
    "Grand Slams": "level0",
    "Masters": "level1"
}

procesar_filtros_y_guardar(
    filtro_menu_id="levelhead",
    filtro_id_map=filtro_id_map_torneos,
    carpeta_salida="ultimas 52 semanas",
    nombre_base="estadisticas_torneos"
)


Procesando filtro: Grand Slams
Guardado: ultimas 52 semanas\estadisticas_torneos_Grand_Slams.csv
Procesando filtro: Masters
Guardado: ultimas 52 semanas\estadisticas_torneos_Masters.csv


In [ ]:
def extraer_estadisticas_personalizadas(tipo_filtro: str, filtro_id: str, nombre_filtro: str, filtro_tiempo_id: str, output_path: str):
    urls = [
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi",
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi?players=51-100"
    ]

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=chrome_options)

    try:
        tablas_totales = []

        for url in urls:
            driver.get(url)
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "spanhead")))

            # Aplicar filtro de tiempo
            driver.find_element(By.ID, "spanhead").click()
            time.sleep(0.5)
            driver.find_element(By.ID, filtro_tiempo_id).click()
            time.sleep(2)

            # Aplicar filtro dinámico (superficie o torneo)
            filtro_categoria_head = {
                "superficie": "surfacehead",
                "torneo": "levelhead",
                "tiempo": "spanhead",  # No se usa como filtro externo aquí, pero se puede ampliar
                "ronda": "roundhead",
                "sets": "setshead"
            }

            driver.find_element(By.ID, filtro_categoria_head[tipo_filtro]).click()
            time.sleep(0.5)
            driver.find_element(By.ID, filtro_id).click()
            time.sleep(3)

            # Aplicar los 4 subfiltros de estadísticas (Serve, Return, Breaks, More)
            secciones_stats = ["statso", "statsw", "statsl", "statst"]
            df_combinado = None

            for stat_id in secciones_stats:
                try:
                    driver.find_element(By.CLASS_NAME, stat_id).click()
                    time.sleep(2)

                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    tabla = soup.find("table", {"id": "matches"})

                    if tabla:
                        rows = tabla.find_all("tr")
                        data = []
                        for row in rows:
                            cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                            if cols:
                                data.append(cols)
                        if not data:
                            continue
                        headers = data[0]
                        data_rows = data[1:-1]  # Excluir la última fila

                        df = pd.DataFrame(data_rows, columns=headers)

                        if df_combinado is None:
                            df_combinado = df
                        else:
                            df_combinado = df_combinado.merge(df, on="Rk", suffixes=("", "_dup"), how="left")

                except Exception as e:
                    print(f"Error en sección {stat_id} para filtro {nombre_filtro}: {e}")

            if df_combinado is not None:
                tablas_totales.append(df_combinado)

        df_final = pd.concat(tablas_totales, axis=0, ignore_index=True)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_final.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"[✓] Guardado: {output_path}")

    finally:
        driver.quit()


In [20]:
# Tipo de filtro: puede ser 'superficie', 'torneo', 'tiempo'
tipo_filtro = "sets"  # <-- Cambia esto a "superficie" si lo deseas

# Diccionarios de ID y nombres visibles en el menú lateral de la web
filtros = {
    "superficie": {
        "surface0": "Hard",
        "surface1": "Clay",
        "surface2": "Grass"
    },
    "torneo": {
        "level0": "GrandSlams",
        "level1": "Masters",
    },
    "tiempo": {
        "span2025qq": "2025",
        "span2024qq": "2024",
        "span2023qq": "2023"
    },
    "ronda": {
        "round0": "final",
        "round1": "semis",
        "round2": "cuartos",
        "round3": "r16",
        "round4": "r32",
        "round5": "r64",
        "round6": "r128"
    },
    "sets": {
        "sets6": "best_of_3"
    }
    
}

# Filtro de tiempo que se aplica siempre (en este ejemplo: 2024)
filtro_tiempo_id = "span2024qq"

# Carpeta donde se guardan todos los CSVs
carpeta = "Estadisticas_2024"
os.makedirs(carpeta, exist_ok=True)

for filtro_id, nombre in filtros[tipo_filtro].items():
    ruta_csv = os.path.join(carpeta, f"stats_{nombre}.csv")
    extraer_estadisticas_personalizadas(tipo_filtro, filtro_id, nombre, filtro_tiempo_id, ruta_csv)



KeyError: 'sets'